In [1]:
import pandas as pd
from Preprocessing.MECO_data_split import concat_MECO_langs

ru = concat_MECO_langs(['ru'])

In [5]:
ru

,Text_ID,Fix_X,Fix_Y,Fix_Duration,Word_Number,Word,Sentence,Language,SubjectID,L2_spelling_skill,...,vocab.t2.5,L2_lexical_skill,TOWRE_word,TOWRE_nonword,motiv,IQ,Age,Sex,Target_Ave,Target_Label
0,1,79,104,221,1,Samuel,"Samuel Morse, best k",ru,rum_1,4.431818,...,4.5,4.0625,3.798077,4.523810,3.7,7.0,30,0,4.144367,4.0
1,1,118,101,289,1,Samuel,"Samuel Morse, best k",ru,rum_1,4.431818,...,4.5,4.0625,3.798077,4.523810,3.7,7.0,30,0,4.144367,4.0
2,1,247,92,337,2,"Morse,","Samuel Morse, best k",ru,rum_1,4.431818,...,4.5,4.0625,3.798077,4.523810,3.7,7.0,30,0,4.144367,4.0
3,1,336,118,281,3,best,"Samuel Morse, best k",ru,rum_1,4.431818,...,4.5,4.0625,3.798077,4.523810,3.7,7.0,30,0,4.144367,4.0
4,1,416,125,250,4,known,"Samuel Morse, best k",ru,rum_1,4.431818,...,4.5,4.0625,3.798077,4.523810,3.7,7.0,30,0,4.144367,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184786,12,388,585,151,145,connectivity,This fusion of capab,ru,ru_55,4.318182,...,3.0,3.7500,4.759615,4.603175,3.4,8.0,29,1,3.605162,4.0
184787,12,480,585,170,145,connectivity,This fusion of capab,ru,ru_55,4.318182,...,3.0,3.7500,4.759615,4.603175,3.4,8.0,29,1,3.605162,4.0
184788,12,589,584,196,146,in,This fusion of capab,ru,ru_55,4.318182,...,3.0,3.7500,4.759615,4.603175,3.4,8.0,29,1,3.605162,4.0
184789,12,643,588,147,147,personal,This fusion of capab,ru,ru_55,4.318182,...,3.0,3.7500,4.759615,4.603175,3.4,8.0,29,1,3.605162,4.0


In [1]:
from Preprocessing.MECO_data_split import MECODataSplit
from sklearn.model_selection import train_test_split
from keras import Sequential
from keras.layers import Conv1D, Dropout, MaxPooling1D, Flatten, Dense
from keras.utils import to_categorical
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

In [23]:
dataset = MECODataSplit(['ru'])
X, y = dataset.split_by_unique_values(split_cols=['SubjectID', 'Text_ID'],
                                      include_cols='fix',
                                      test_size=0, fix_threshold=0,
                                      resample='truncate', series_length=200)

In [24]:
print(X.shape, y.shape)

(331, 200, 3) (331, 1)


In [25]:
y = to_categorical(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [26]:
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]

In [27]:
def train_model(conv_filters=64, kernel_size=3, strides=1, dropout_rate=0.5, pool_size=2, epochs=10, batch_size=32):
    model = Sequential()
    model.add(Conv1D(filters=conv_filters, kernel_size=kernel_size, strides=strides,
                     activation='relu', input_shape=(n_timesteps, n_features)))
    model.add(Conv1D(filters=conv_filters, kernel_size=kernel_size, strides=strides, activation='relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Dropout(dropout_rate))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)

    return model

In [34]:
cnn = train_model(epochs=8, batch_size=32, dropout_rate=0.5, conv_filters=64, kernel_size=3)

Epoch 1/8
9/9 [==============================] - 1s 10ms/step - loss: 301.1114 - accuracy: 0.4432
Epoch 2/8
9/9 [==============================] - 0s 9ms/step - loss: 88.0619 - accuracy: 0.5227
Epoch 3/8
9/9 [==============================] - 0s 9ms/step - loss: 48.5254 - accuracy: 0.4394
Epoch 4/8
9/9 [==============================] - 0s 12ms/step - loss: 14.9311 - accuracy: 0.6364
Epoch 5/8
9/9 [==============================] - 0s 22ms/step - loss: 6.4151 - accuracy: 0.6705
Epoch 6/8
9/9 [==============================] - 0s 16ms/step - loss: 2.1082 - accuracy: 0.7841
Epoch 7/8
9/9 [==============================] - 0s 11ms/step - loss: 1.3995 - accuracy: 0.7614
Epoch 8/8
9/9 [==============================] - 0s 12ms/step - loss: 0.9329 - accuracy: 0.7841


In [35]:
loss, accuracy = cnn.evaluate(X_train, y_train)

9/9 [==============================] - 0s 3ms/step - loss: 0.2663 - accuracy: 0.8864
